# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.57,80,100,1.79,NZ,1707671325
1,1,port-aux-francais,-49.3500,70.2167,8.58,98,55,17.08,TF,1707671367
2,2,puerto natales,-51.7236,-72.4875,9.25,66,40,9.26,CL,1707671368
3,3,hasaki,35.7333,140.8333,5.16,87,98,4.25,JP,1707671368
4,4,stanley,54.8680,-1.6985,5.77,89,20,5.14,GB,1707671368


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,  
    size="Humidity",
    color = "City",
    alpha=0.6
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df=city_data_df[(city_data_df['Max Temp']>=20 ) & (city_data_df["Wind Speed"] < 4.5) &(city_data_df['Max Temp']<30 ) & (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
city_data_df=city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,cruz grande,16.7333,-99.1333,29.09,47,0,1.74,MX,1707671371
77,77,haiku-pauwela,20.9219,-156.3051,20.02,88,0,1.54,US,1707671377
111,111,cabo san lucas,22.8909,-109.9124,20.90,62,0,0.45,MX,1707671180
127,127,sittwe,20.1500,92.9000,20.43,62,0,4.41,MM,1707671383
172,172,faya-largeau,17.9257,19.1043,27.70,9,0,3.73,TD,1707671389
173,173,nelson bay,-32.7167,152.1500,22.03,89,0,1.34,AU,1707671389
224,224,adrar,20.5022,-10.0711,29.31,28,0,3.59,MR,1707671399
225,225,geraldton,-28.7667,114.6000,21.23,88,0,3.09,AU,1707671051
369,369,port hedland,-20.3167,118.5667,27.03,94,0,3.09,AU,1707671428
383,383,san luis de la loma,17.2706,-100.8939,27.70,58,0,2.65,MX,1707671430


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=pd.DataFrame()
hotel_df.loc[:,"City"]=city_data_df.loc[:,"City"]
hotel_df.loc[:,"Country"]=city_data_df.loc[:,"Country"]
hotel_df.loc[:,"Lat"]=city_data_df.loc[:,"Lat"]
hotel_df.loc[:,"Lng"]=city_data_df.loc[:,"Lng"]
hotel_df.loc[:,"Humidity"]=city_data_df.loc[:,"Humidity"]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name']=""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
29,cruz grande,MX,16.7333,-99.1333,47,
77,haiku-pauwela,US,20.9219,-156.3051,88,
111,cabo san lucas,MX,22.8909,-109.9124,62,
127,sittwe,MM,20.1500,92.9000,62,
172,faya-largeau,TD,17.9257,19.1043,9,
173,nelson bay,AU,-32.7167,152.1500,89,
224,adrar,MR,20.5022,-10.0711,28,
225,geraldton,AU,-28.7667,114.6000,88,
369,port hedland,AU,-20.3167,118.5667,94,
383,san luis de la loma,MX,17.2706,-100.8939,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
limit=20
params = {
    "apiKey":geoapify_key,
    "limit":limit,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,'Lat']
    longitude=hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cruz grande - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
cabo san lucas - nearest hotel: Comfort Rooms
sittwe - nearest hotel: Yuzana Aung Motel 1
faya-largeau - nearest hotel: No hotel found
nelson bay - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
port hedland - nearest hotel: The Esplanade Hotel
san luis de la loma - nearest hotel: No hotel found
vallenar - nearest hotel: Hotel del Marqués
duong dong - nearest hotel: Kim Hông Ngoc
veracruz - nearest hotel: No hotel found
phang nga - nearest hotel: Hotel
sultanah - nearest hotel: فندق جلنار
manzanillo - nearest hotel: Hotel Venus
port augusta - nearest hotel: Hotel Commonwealth
aswan - nearest hotel: Yaseen hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
29,cruz grande,MX,16.7333,-99.1333,47,No hotel found
77,haiku-pauwela,US,20.9219,-156.3051,88,Inn At Mama's Fish House
111,cabo san lucas,MX,22.8909,-109.9124,62,Comfort Rooms
127,sittwe,MM,20.1500,92.9000,62,Yuzana Aung Motel 1
172,faya-largeau,TD,17.9257,19.1043,9,No hotel found
173,nelson bay,AU,-32.7167,152.1500,89,No hotel found
224,adrar,MR,20.5022,-10.0711,28,No hotel found
225,geraldton,AU,-28.7667,114.6000,88,Ocean Centre Hotel
369,port hedland,AU,-20.3167,118.5667,94,The Esplanade Hotel
383,san luis de la loma,MX,17.2706,-100.8939,58,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,  
    size="Humidity",
    color = "City",
    alpha=0.6,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)